<a href="https://colab.research.google.com/github/CristianEyebrow/CSCI_167/blob/FinalProject/167_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installations
!pip install -q torch torchvision pandas
# jupyter nbconvert --ClearMetadataPreprocessor.enabled=True --inplace --to notebook filename.ipynb

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

from tqdm.auto import tqdm

# For plots
%matplotlib inline

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [ ]:
# Training data
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2470, 0.2435, 0.2616)),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2470, 0.2435, 0.2616)),
])

# Load base training dataset with NO transform yet
base_train_dataset = datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=None  # we'll add transforms in a wrapper
)

# Test dataset
test_dataset = datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=test_transform
)

# Train/validation split
num_train = len(base_train_dataset)
val_size = int(0.1 * num_train)
train_size = num_train - val_size

indices = np.random.permutation(num_train)
val_indices = indices[:val_size]
train_indices = indices[val_size:]

class CIFAR10Subset(Dataset):
    """
    A thin wrapper to:
    - select a subset of indices from a base CIFAR-10 dataset
    - apply a given transform to the images
    """
    def __init__(self, base_dataset, indices, transform=None):
        self.base_dataset = base_dataset
        self.indices = list(indices)
        self.transform = transform

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        base_idx = self.indices[idx]
        img, label = self.base_dataset[base_idx]  # img is PIL image
        if self.transform is not None:
            img = self.transform(img)
        return img, label

# Create train/val subsets that share the same data but use different transforms
train_dataset = CIFAR10Subset(base_train_dataset, train_indices, transform=train_transform)
val_dataset   = CIFAR10Subset(base_train_dataset, val_indices,   transform=test_transform)

from torch.utils.data import DataLoader

def make_dataloaders(batch_size=128):
    # Avoids multiprocessing issues
    num_workers = 0

    pin_memory = (device.type == "cuda")

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=pin_memory,
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=pin_memory,
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=pin_memory,
    )

    return train_loader, val_loader, test_loader

print(f"Train size: {len(train_dataset)}, Val size: {len(val_dataset)}, Test size: {len(test_dataset)}")


In [ ]:
# Simple and Deep CNN
class SimpleCNN(nn.Module):
    """
    A small CNN for CIFAR-10:
    - 2 convolutional layers
    - 2 fully-connected layers
    """
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # [B, 32, 32, 32]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                             # [B, 32, 16, 16]
            nn.Conv2d(32, 64, kernel_size=3, padding=1), # [B, 64, 16, 16]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)                              # [B, 64, 8, 8]
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 8 * 8, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


class DeeperCNN(nn.Module):
    """
    A deeper CNN with:
    - 3 convolutional blocks
    - BatchNorm
    - Dropout for regularization
    """
    def __init__(self, num_classes=10, dropout_p=0.5):
        super().__init__()
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            # Block 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            # Block 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(dropout_p),
            nn.Linear(256 * 4 * 4, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_p),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [ ]:
# Training and evaluations
def accuracy_from_logits(logits, targets):
    preds = torch.argmax(logits, dim=1)
    correct = (preds == targets).sum().item()
    total = targets.size(0)
    return correct / total

def train_one_epoch(model, optimizer, train_loader, criterion, device):
    model.train()
    running_loss = 0.0
    running_acc = 0.0
    total = 0

    for images, labels in tqdm(train_loader, leave=False):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        batch_size = labels.size(0)
        running_loss += loss.item() * batch_size
        running_acc += accuracy_from_logits(outputs, labels) * batch_size
        total += batch_size

    epoch_loss = running_loss / total
    epoch_acc = running_acc / total
    return epoch_loss, epoch_acc

@torch.no_grad()
def evaluate(model, data_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_acc = 0.0
    total = 0

    for images, labels in data_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        batch_size = labels.size(0)
        running_loss += loss.item() * batch_size
        running_acc += accuracy_from_logits(outputs, labels) * batch_size
        total += batch_size

    epoch_loss = running_loss / total
    epoch_acc = running_acc / total
    return epoch_loss, epoch_acc

def create_model(model_name):
    if model_name == "SimpleCNN":
        return SimpleCNN(num_classes=10)
    elif model_name == "DeeperCNN":
        return DeeperCNN(num_classes=10, dropout_p=0.5)
    else:
        raise ValueError(f"Unknown model_name: {model_name}")

def create_optimizer(optimizer_name, model_parameters, lr, weight_decay=0.0, momentum=0.9):
    if optimizer_name == "SGD":
        return optim.SGD(model_parameters, lr=lr, momentum=momentum, weight_decay=weight_decay)
    elif optimizer_name == "Adam":
        return optim.Adam(model_parameters, lr=lr, weight_decay=weight_decay)
    else:
        raise ValueError(f"Unknown optimizer_name: {optimizer_name}")

def create_scheduler(scheduler_name, optimizer, step_size=10, gamma=0.1):
    if scheduler_name is None or scheduler_name.lower() == "none":
        return None
    elif scheduler_name == "StepLR":
        return optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
    else:
        raise ValueError(f"Unknown scheduler_name: {scheduler_name}")

def run_experiment(config):
    """
    config: dict with keys:
      - name
      - model_name
      - optimizer_name
      - lr
      - batch_size
      - weight_decay
      - scheduler_name
      - num_epochs
    """
    print("=" * 80)
    print(f"Starting experiment: {config['name']}")
    print(config)

    # Data loaders
    train_loader, val_loader, test_loader = make_dataloaders(batch_size=config["batch_size"])

    # Model, loss, optimizer, scheduler
    model = create_model(config["model_name"]).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = create_optimizer(
        optimizer_name=config["optimizer_name"],
        model_parameters=model.parameters(),
        lr=config["lr"],
        weight_decay=config.get("weight_decay", 0.0),
        momentum=config.get("momentum", 0.9)
    )
    scheduler = create_scheduler(
        scheduler_name=config.get("scheduler_name", None),
        optimizer=optimizer,
        step_size=config.get("step_size", 10),
        gamma=config.get("gamma", 0.1)
    )

    num_epochs = config["num_epochs"]

    history = {
        "train_loss": [],
        "val_loss": [],
        "train_acc": [],
        "val_acc": [],
    }

    best_val_acc = 0.0
    best_epoch = -1

    for epoch in range(num_epochs):
        print(f"\nEpoch [{epoch+1}/{num_epochs}]")

        train_loss, train_acc = train_one_epoch(
            model=model,
            optimizer=optimizer,
            train_loader=train_loader,
            criterion=criterion,
            device=device
        )

        val_loss, val_acc = evaluate(
            model=model,
            data_loader=val_loader,
            criterion=criterion,
            device=device
        )

        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["train_acc"].append(train_acc)
        history["val_acc"].append(val_acc)

        if scheduler is not None:
            scheduler.step()

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_epoch = epoch

        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%")
        print(f"Val   Loss: {val_loss:.4f}, Val   Acc: {val_acc*100:.2f}%")

    # Final test evaluation
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    print(f"\nFinal Test Loss: {test_loss:.4f}, Test Acc: {test_acc*100:.2f}%")
    print(f"Best Val Acc: {best_val_acc*100:.2f}% at epoch {best_epoch+1}")

    result_summary = {
        "experiment_name": config["name"],
        "model": config["model_name"],
        "optimizer": config["optimizer_name"],
        "batch_size": config["batch_size"],
        "lr": config["lr"],
        "weight_decay": config.get("weight_decay", 0.0),
        "scheduler": config.get("scheduler_name", "none"),
        "num_epochs": num_epochs,
        "best_val_acc": best_val_acc,
        "test_acc": test_acc,
    }

    return result_summary, history

def plot_learning_curves(history, title="Learning Curves"):
    epochs = range(1, len(history["train_loss"]) + 1)

    plt.figure(figsize=(12, 4))

    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history["train_loss"], label="Train Loss")
    plt.plot(epochs, history["val_loss"],   label="Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(title + " - Loss")
    plt.legend()

    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history["train_acc"], label="Train Acc")
    plt.plot(epochs, history["val_acc"],   label="Val Acc")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title(title + " - Accuracy")
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
# Hyperparameters
NUM_EPOCHS = 15

experiments = [
    {
        "name": "SimpleCNN_SGD_bs128_nosched",
        "model_name": "SimpleCNN",
        "optimizer_name": "SGD",
        "lr": 0.01,
        "batch_size": 128,
        "weight_decay": 0.0,
        "scheduler_name": None,
        "num_epochs": NUM_EPOCHS,
    },
    {
        "name": "SimpleCNN_SGD_wd5e-4_stepLR",
        "model_name": "SimpleCNN",
        "optimizer_name": "SGD",
        "lr": 0.01,
        "batch_size": 128,
        "weight_decay": 5e-4,
        "scheduler_name": "StepLR",
        "step_size": 3,
        "gamma": 0.5,
        "num_epochs": NUM_EPOCHS,
    },
    {
        "name": "SimpleCNN_Adam_bs64",
        "model_name": "SimpleCNN",
        "optimizer_name": "Adam",
        "lr": 0.001,
        "batch_size": 64,
        "weight_decay": 0.0,
        "scheduler_name": None,
        "num_epochs": NUM_EPOCHS,
    },
    {
        "name": "DeeperCNN_Adam_bs128_wd5e-4_stepLR",
        "model_name": "DeeperCNN",
        "optimizer_name": "Adam",
        "lr": 0.001,
        "batch_size": 128,
        "weight_decay": 5e-4,
        "scheduler_name": "StepLR",
        "step_size": 3,
        "gamma": 0.5,
        "num_epochs": NUM_EPOCHS,
    },
]


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Experiment and results
all_results = []
all_histories = {}

for config in experiments:
    result_summary, history = run_experiment(config)
    all_results.append(result_summary)
    all_histories[config["name"]] = history

# Convert results for comparison
results_df = pd.DataFrame(all_results)
results_df["best_val_acc_%"] = results_df["best_val_acc"] * 100
results_df["test_acc_%"] = results_df["test_acc"] * 100

print("\n===== Summary of Experiments =====")
display(results_df[[
    "experiment_name",
    "model",
    "optimizer",
    "batch_size",
    "lr",
    "weight_decay",
    "scheduler",
    "num_epochs",
    "best_val_acc_%",
    "test_acc_%"
]])

In [ ]:
# Plots
for exp_name, hist in all_histories.items():
    plot_learning_curves(hist, title=exp_name)